## Generator
When we have an "infinite" source to generate data from, no container will be suitable. Then, we have a "generator".
Generator has generator function and generator expressions. 

1. Generator Expression: `for i in <generator>`. You can call the built-in `next()` on the generator, the generator will keep spitting out values until finishing the last value. Then, `StopIteration` is called. 
2. For loop calls next(iter(iterable)). 

In [39]:
g = (x for x in range(10))
print(next(g))   # this is totes valid
print("===============")
# can keep on iterating 
for i in g: 
    print(i)

# use for loop 
bday_gen = Bday_Gen()
print("===============")
for b in bday_gen: 
    print(b)

0
1
2
3
4
5
6
7
8
9
1
2
Print after yield
123, 456


1. Generators function are functions with `yield`` inside. How it works:
    - Go to first yield, 
    - in the subsequent yield, start from the next line, finish at the next yield. 
    - By default, `StopIteration` exception when the last value is yielded. So you don't have to write this explicitly
2. The generator object has `__next__()`, which makes it an iterator


In [40]:
def Bday_Gen():
    yield 1
    yield 2
    # 1. get a StopIteration exception RIGHT after the last yield
    # Then, the rest of the function after the last yield will be executed before the main function continues.
    print("Print after yield")
    print("123, 456")

bday_gen = Bday_Gen()
while (True):
    try:
        print(next(bday_gen))
        print("while loop check")
    except StopIteration:
        break
print("Done While loop")


1
while loop check
2
while loop check
Print after yield
123, 456
Done While loop


### Iterable, Iterator and Generator
A container (list, tuple, etc.) are iterables. Iterator Protocol: 
    - `container.__iter__() -> Iterator` defines an iterable. `__iter__()` is used for `for ... in`
    - For `Iterator`, we need 
        - `Iterator.__iter__() -> self`, for `for ... in`. 
        - `__next__()` that raises `StopIteration`. 
A generator function / expression automatically synthesizes `__iter()__` and `__next__()`

In [41]:
ls = [1,2,3]
try: 
    next(ls)
except TypeError:
    print("ls is not an iterator, it's a container (iterable). Use iter()")
i = iter(ls)
next(i)

ls is not an iterator, it's a container (iterable). Use iter()


1

Iterables are: 
    - `range` (`iter(range)`return an iterator.)
    - generator
    - containers

In [42]:
i = iter(range(3))
print(next(i))
print(next(i))

0
1


### Design Patterns for generators
Generator expression is an elegant way to reduce and transform data. max(), sum(), join(), no need to create a list. 
Generator Function can be used to: 
    - iterating over N threads and get their output. But we don't have GIL here, so this could be faster than a lock 
    - generate output indefinitely, (real time hardware)
    - search which decouples search process from the upper stream code

In [43]:
# 5 generator is a good design pattern for search
ls = [1,2,3,4,5,6,7,8, 2, 2]
def search(num): 
    for n in ls: 
        if n == num: 
            yield n
for s in search(2): 
    print(s)

s = ('ACME', 50, 123.45)
# ","("abc") = "a,b,c"
print(",".join(str(x) for x in s))


portfolio = [ {'name':'GOOG', 'shares': 50}, {'name':'YHOO', 'shares': 75}, {'name':'AOL', 'shares': 20}]
print(min(p["shares"] for p in portfolio))
nums = [1,2,3,45]
print("sum using generator: ", sum(n for n in nums))

2
2
2
ACME,50,123.45
20
sum using generator:  51


## Yield From (python 3.3+)
1. This expression actually means "Yield from iteratble". Function with `Yield from` is still a generator. It's  roughly equivalent to
    ```
    yield from Iteratble == 
    for i in Iterable: 
        yield i
    ```

In [44]:
# Example 1: yield from range object
def yield_from_gen():
    for i in range(2):
        yield i

for i in yield_from_gen():
    print(i)

# Example 2: yield from a generator
def test(n): 
    i = 0
    while i < n: 
        print("test i: ", i)
        yield i
        i+=1
    print("This will be executed after the yield")

def yield_from_func(n): 
    print("Start: ")
    j = yield from test(n)
    print("j: ", j)
    # 'This will be executed after the yield' will STILL be executed
    # The last value of j will be None, after 
    print("========")
    # You can have another round 
    yield from test(n)
    print("End: ")

for i in yield_from_func(4): 
    print("main loop i: ", i)

0
1
Start: 
test i:  0
main loop i:  0
test i:  1
main loop i:  1
test i:  2
main loop i:  2
test i:  3
main loop i:  3
This will be executed after the yield
j:  None
test i:  0
main loop i:  0
test i:  1
main loop i:  1
test i:  2
main loop i:  2
test i:  3
main loop i:  3
This will be executed after the yield
End: 


2. `Yield from` is to address the woes in `yield`. If you have `return` after a `yield`, the returned value will trigger an implicit `StopIteration`, but its value won't be captured.  

In [45]:
# 1
def some_gen(): 
    yield 0
    yield 1
    # You can also do return
    return "Done0"

g = some_gen()
print(next(g))
# now see 1, because generator picks up where it left off.
print(next(g))
try:
    print(next(g))
except StopIteration:
    print("stop iteration is received, no return value")

# 2
print("If directly using for loop, return value won't be returned ")
g = some_gen()
for i in g: 
    print(i)

print("If using yield from, then using for loop, return value WILL be returned")
def test_some_gen():
    g = some_gen()
    res = yield from g
    print(res)
    print("After everything in yield from is ended, what's after will be finished")
for i in test_some_gen():
    print(i)

# 3
tsg = test_some_gen()
print(next(tsg))
print(next(tsg))
try: 
    next(tsg) # this will finish the part after yield from, but will also raise stopiteration
except StopIteration:
    print("If you manually call next() just enough times to finish yielding, you will see the return value, and the rest of the generator")
    


0
1
stop iteration is received, no return value
If directly using for loop, return value won't be returned 
0
1
If using yield from, then using for loop, return value WILL be returned
0
1
Done0
After everything in yield from is ended, what's after will be finished
0
1
Done0
After everything in yield from is ended, what's after will be finished
If you manually call next() just enough times to finish yielding, you will see the return value, and the rest of the generator



3. `yield from` creates a layer to interact with the "sender", as a pipe. Python calls it a "proxy generator"
    - Advance uses of `yield from`: can do `send`, `.throw()` and `close()` (see below).

In [61]:
def total_average():
    total = 0.0
    count = 0
    avg = None
    print("starting average generator")
    while True:
        num = yield avg
        if num is None:
            break
        total += num
        count += 1
        avg = total/count

def wrap_average(average_generator):
    """This is just a pipe to the generator"""
    print("starting wrap_average")
    avg = yield from average_generator

# Note: total_average() is the generator object. total_average is generator function
w = wrap_average(total_average())
# Execute everthing until hitting the first iterator. None being returned
print("starting the generator: ", next(w))
print(w.send(3))
print(w.send(4))
# Finish Coroutine
# ? Not sure why w.send(None) is giving me stop iteration?
# w.send(None)
try: 
    if w.send(5) == 4:
        w.throw(ValueError("I don't like this value"))
except ValueError:
    pass
w.close()
        
    

starting wrap_average
starting average generator
starting the generator:  None
3.0
3.5


## Coroutines 
Coroutines can be thought of as a function that can be "paused"